In [ ]:
%%html
<style>
@import url("https://fonts.googleapis.com/css2?family=Dancing+Script&display=swap");
@import url('https://fonts.googleapis.com/css2?family=Fuggles&display=swap');
.journal {
  background-color: #f8f5de;
  background-image: linear-gradient(to right, rgba(255,210,0,0.4), rgba(200, 160, 0, 0.1) 11%, rgba(0,0,0,0) 35%, rgba(200, 160, 0, 0.1) 65%);
  box-shadow: inset 0 0 75px rgba(255,210,0,0.3), inset 0 0 20px rgba(255,210,0,0.4), inset 0 0 30px rgba(220,120,0,0.8);
  color: rgba(0,0,0,0.5);
  font-size: 1.5em;
  font-family: "Dancing Script", cursive;
  letter-spacing: 0.05em;
  line-height: 1.1;
  padding: 1em 2em 1em 2em;
  width: initial;
  max-width: 20em;
}
.journal_top {
  font-size: 1.5em;
  font-family: "Dancing Script", cursive;
  letter-spacing: 0.05em;
  line-height: 1.1;
  padding: 1em 2em 1em 2em;
}
.date {
    color: rgba(50,10,10,0.5);
    margin-bottom: 1em;
}
.fancycaps {
  color: rgba(0,0,0,0.7);
  float:left;
  font-size: 7em;
  line-height: 60px;
  padding-right: 12px;
  position:relative;
  top:8px;
}
.signature {
    width: 100%;
    text-align: right;
    font-size: 2em;
    font-style: italic;
    font-family: "Fuggles", cursive;
    margin-top: 1em;
}
</style>
<div class="journal_top">
<h1>Learn Cyclone DDS Python, Chapter 2</h1>
<h4>with Captain Corsaro</h4>
</div>

<div class="journal">
    <div class="date">June 20th, 1674</div>
    <p>Today we made landing on the first island, which I have dubbed Partecipante. Small but plentiful in coconuts, dry wood and root vegetables. The presence of coconuts on this island without any proof of human intervention tells me we are definitely not in the mediterranean anymore, if that wasn't clear from the days travel without land in sight before.</p>
    
    <p>One of the Dutch crewmates was commenting on the different coconut palm variaties, apparantly amazed by the great local variety in "koQoSnoten".</p>
    
    <div class="signature">Captain A. Corsaro</div>
</div>

This is chapter 2 of the Cyclone DDS Python tutorial, where we will dive into Qos, Listeners and WaitSets. The basics of topics, readers and writers were explained in [chapter 1](CycloneTutorialChapter1.ipynb) and you can always go back to it.

In [ ]:
from questing import Journal

journal = Journal(seed=None, chapter=2)
print(journal.seed)

## koQoSnoten

QoS stands for Quality of Service. In DDS you can apply a set of QoS policies to any entity, controlling the behaviour of the DDS system. DDS cannot know which data is critically important and which is actually irrelevant after a few seconds, so QoS allows you to tell DDS the meta-data around data delivery, lifetime and storage. All policies on the writer and reader side together form the "contract" that DDS will have to furfill. Here some user discretion is adviced: it is possible to cause incompatibilities between reader and writer with some of the QoS policies, creating a contract that is impossible for DDS to furfill.

In [ ]:
quest = journal.quest("koqosnoot")
quest.prompt()
quest.start()

from cyclonedds.core import Qos, Policy
from cyclonedds.util import duration

# Create a Shared or Exclusive Ownership policy. They take no arguments.
# The solution is under quest.hint(index=0)
policy0 = Policy.Ownership.Shared
quest.check("policy-0", policy0)

# Create an Automatic Liveliness policy. It takes a lease duration as argument,
# make that two days. The solution is under quest.hint(index=1)
policy1 = Policy.Liveliness.Automatic(duration(days=2))
quest.check("policy-1", policy1)

# Create a Qos object with a Reliable Reliability policy, taking a max blocking
# time as argument, set that to 250 milliseconds. Also add a KeepAll History
# policy, it takes no arguments. The solution is under quest.hint(index=2)
qos0 = Qos(Policy.Reliability.Reliable(duration(milliseconds=250)), Policy.History.KeepAll)
quest.check("koqosnoot-0", qos0)

# Create any Qos object with 4 policies.
# An example solution is under quest.hint(index=3)
qos1 = Qos(
    Policy.Reliability.BestEffort,
    Policy.Durability.TransientLocal,
    Policy.History.KeepAll,
    Policy.Partition(["a", "b"])
)
quest.check("koqosnoot-1", qos1)

# Create a Qos object with 5 policies by adding one to the previous qos
# You can inherit other Qos by giving the keyword argument 'base':
#    qos = Qos(policy1, ..., base=other_qos)
# An example solution is under quest.hint(index=4)
qos2 = Qos(Policy.TransportPriority(4), base=qos1)
quest.check("koqosnoot-2", qos2)

# Example: you can inspect any QoS object by iteration
for policy in qos2:
    print(policy)

quest.finish()

In [ ]:
## Hey! Listen!

<div class="journal">
    <div class="date">June 21th, 1674</div>
    <p>I've always said that in order for your crew to stay alert and efficient you need to make sure they get enough rest. The entire crew is off today for some shore leave, drinking from coconuts and listening to the waves. Tomorrow we shall sail past the other islands and see what secrets they hold.</p>
</div>

Listeners are a callback mechanism, allowing you to listen to what is going on inside the Cyclone DDS middleware. While a networking infrastructure that is totally abstracted away is nice, sometimes it is necessary to react to something happening on the other side. For example, publishing complex computations while no-one is listening is a waste of CPU-cycles.

A Cyclone DDS `Listener` has the following callbacks: `on_data_available`, `on_inconsistent_topic`, `on_liveliness_lost`, `on_liveliness_changed`, `on_offered_deadline_missed`, `on_offered_incompatible_qos`, `on_data_on_readers`, `on_sample_lost`, `on_sample_rejected`, `on_requested_deadline_missed`, `on_requested_incompatible_qos`, `on_publication_matched` and `on_subscription_matched`.

In [ ]:
quest = journal.quest("hey-listen")
quest.prompt()
quest.start()

from pycdr import cdr
from cyclonedds.core import Listener, Qos, Policy
from cyclonedds.domain import DomainParticipant
from cyclonedds.topic import Topic
from cyclonedds.sub import DataReader
from cyclonedds.util import duration
from threading import Event

@cdr
class Wave:
    height: int
    loudness: float

quest.check("wave", Wave)

event = Event()
class MyListener(Listener):
    def on_data_available(self, reader):
        event.set()

participant = DomainParticipant()
topic = Topic(participant, "Wave", Wave)
mylistener = MyListener()
myqos = Qos(Policy.Reliability.Reliable(duration(milliseconds=200)))
reader = DataReader(participant, topic, qos=myqos, listener=mylistener)
quest.check("reader", reader)

event.wait()

for wave in reader.read_iter(timeout=duration(milliseconds=200)):
    print(wave)
    quest.check("a-lovely-wave", wave)

quest.finish()

In [ ]:
## Waiting for the tide

<div class="journal">
    <div class="date">June 22th, 1674</div>
    <p>As soon as the tide comes in we are ready to leave.</p>
</div>

Waitsets allow you to wait for a set of DDS entities. It greatly simplifies blocking execution till one of multiple things trigger.